# Usando MongoDB con Python
## Una Historia de Esperanza y Supervivencia

### Pablo Toledo

Twitter: [@t_papablo](https://twitter.com/t_papablo)

Telegram: [papablot](t.me/papablot)

# A long time ago...

* Necesidad de almacenar información
* Los atributos y estructura podían cambiar
* Podíamos aprender algo nuevo :)

# MongoDB

* Orientada a Documentos
* Habla en términos de JSON

# MongoDB con Python

* [Mongoengine](mongoengine.org)
    * ODM (Object Document Mapper)
![mongoengine](./mongoengine.png)

* Soporte para varios frameworks

![soportes](./mongoengine-cosas.png)

![django](./django-mongo.png)

![flask](./flask-mongo.png)

In [85]:
import mongoengine

In [86]:

# Connect
mongo_connection = {
    'db':'mongopy_db',
    'username' : 'mongopy',
    'password' : 'm0ng0py',
    'host' : 'localhost',
    'port':27020,
    'alias' : 'default'
}

mongoengine.connect(
    db=mongo_connection['db'],
    username=mongo_connection['username'],
    password=mongo_connection['password'],
    host=mongo_connection['host'],
    port=mongo_connection['port'],
    alias=mongo_connection['alias']
)

mongo_connection

{'db': 'mongopy_db',
 'username': 'mongopy',
 'password': 'm0ng0py',
 'host': 'localhost',
 'port': 27020,
 'alias': 'default'}

In [87]:
from mongoengine import Document, fields

In [4]:
class Persona(Document):
    nombre = fields.StringField(max_length=40)
    edad = fields.IntField(min_value=0)
    
    def __str__(self):
        return f'nombre: {self.nombre} | edad: {self.edad}'

In [96]:
persona = Persona(nombre='Pablo',edad=22)
persona.save()

<Persona: nombre: Pablo | edad: 22>

In [97]:
persona.to_json()
from json import loads

loads(persona.to_json())

{'_id': {'$oid': '5c1570b5dbda691225587183'}, 'nombre': 'Pablo', 'edad': 22}

In [98]:
import random

with open('archivito.txt','r') as archivo:
    lineas = archivo.readlines()[0]
    for l in lineas.split():
        Persona(nombre=l.lower(),edad=random.randint(20,40)).save()

In [104]:
# Cueris

Persona.objects.filter(edad=22).filter(nombre='Pablo')

[<Persona: nombre: Pablo | edad: 22>, <Persona: nombre: Pablo | edad: 22>, <Persona: nombre: Pablo | edad: 22>, <Persona: nombre: Pablo | edad: 22>, <Persona: nombre: Pablo | edad: 22>, <Persona: nombre: Pablo | edad: 22>, <Persona: nombre: Pablo | edad: 22>, <Persona: nombre: Pablo | edad: 22>]

In [106]:
# Cueris con objetos Q
from mongoengine import Q

q = Q(edad__lt=35) | Q(nombre__gt='lorem')

Persona.objects.filter(q)

[<Persona: nombre: ipsum | edad: 28>, <Persona: nombre: dolor | edad: 21>, <Persona: nombre: sit | edad: 39>, <Persona: nombre: sadipscing | edad: 28>, <Persona: nombre: elitr | edad: 28>, <Persona: nombre: sed | edad: 31>, <Persona: nombre: nonumy | edad: 21>, <Persona: nombre: eirmod | edad: 34>, <Persona: nombre: tempor | edad: 30>, <Persona: nombre: invidunt | edad: 29>, <Persona: nombre: ut | edad: 35>, <Persona: nombre: labore | edad: 28>, <Persona: nombre: dolore | edad: 29>, <Persona: nombre: magna | edad: 38>, <Persona: nombre: aliquyam | edad: 22>, <Persona: nombre: erat | edad: 27>, <Persona: nombre: sed | edad: 35>, <Persona: nombre: diam | edad: 24>, <Persona: nombre: voluptua | edad: 20>, <Persona: nombre: vero | edad: 26>, '...(remaining elements truncated)...']

In [108]:
Persona.objects.filter(nombre='Pablo').delete()

8

In [112]:
# %load heroes.py
from mongoengine import fields
import mongoengine_goodjson as gj

# Docuementos almacenados en colecciones propias
class Localidad(gj.Document):
    nombre = fields.StringField(max_length=50)

# Documentos que no requieren su propia colección
class Superpoder(gj.EmbeddedDocument):
    nombre = fields.StringField(max_length=100, required=True)
    descripcion = fields.StringField(max_length=300, required=False)

#Documentos con documentos adentro
class Heroe(gj.Document):
    nombre = fields.StringField(max_length=50, required=True)

    superpoder = fields.EmbeddedDocumentField(Superpoder, required=False)
    
    localidad = fields.ReferenceField(Localidad,required=True)
    
    # File fields!
    historia = fields.FileField(required=False)

    def __str__(self):
        return f'{self.nombre} | {self.superpoder.nombre}'
# We have to go deeper
class Liga(gj.Document):
    nombre = fields.StringField(max_length=100)

    lugar = fields.ReferenceField(Localidad, requered=False)

    integrantes = fields.ListField(
        fields.ReferenceField(Heroe),
        required=False
    )


In [121]:
superpoder = Superpoder(nombre='plata', descripcion='mucha plata')

batman = Heroe(nombre='batman',superpoder=superpoder, localidad=Localidad(nombre='Ciudad Gótica').save())

In [122]:
batman.save()

<Heroe: batman | plata>

In [126]:
[h.to_json(follow_reference=True) for h in Heroe.objects.all()]

['{"nombre": "batman", "superpoder": {"nombre": "plata", "descripcion": "mucha plata"}, "localidad": {"nombre": "Ciudad G\\u00f3tica", "id": "5c157301dbda6912255871b8"}, "id": "5c157301dbda6912255871b9"}']

In [128]:
batman

<Heroe: batman | plata>

In [133]:
file_handler = open('archivito.txt','rb')


In [146]:
batman.historia.delete()